In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22.xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구,노인인구,장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수'],
      dtype='object')

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [6]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수'],
      dtype='object')

## 데이터 스케일링

In [7]:
scaler=MinMaxScaler()
df = origin1.copy()
df['인구minmax']=scaler.fit_transform(df[['계_명']])
df['지하철minmax']=scaler.fit_transform(df[['지하철역_수']])
df['주차장면수minmax']=scaler.fit_transform(df[['면수']])
df['버스정류장minmax']=scaler.fit_transform(df[['행정동별버스정류장수']])
df['자전거대여소minmax']=scaler.fit_transform(df[['공공자전거대여소수']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0,3,0.212126,0.000000,0.209007,0.294118,0.065217
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0,12,0.169482,0.285714,0.111957,0.245098,0.260870
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0,5,0.045708,0.000000,0.105931,0.137255,0.108696
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0,5,0.169755,0.000000,0.186172,0.313725,0.108696
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0,5,0.320797,0.000000,0.509990,0.509804,0.108696


## 파생변수(`접근성지수`) 생성

In [8]:
df['접근성지수'] = (df['지하철minmax'] + df['주차장면수minmax'] + df['버스정류장minmax'] + df['자전거대여소minmax']) / df['인구minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,32,0,3,0.212126,0.000000,0.209007,0.294118,0.065217,2.679272
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,27,0,12,0.169482,0.285714,0.111957,0.245098,0.260870,5.331780
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,16,0,5,0.045708,0.000000,0.105931,0.137255,0.108696,7.698424
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,10,34,0,5,0.169755,0.000000,0.186172,0.313725,0.108696,3.585121
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,10,54,0,5,0.320797,0.000000,0.509990,0.509804,0.108696,3.517773


## 파생 변수의 값 정렬

In [9]:
df['접근성지수'].value_counts().sort_index()

접근성지수
0.325775     1
0.419616     1
0.468057     1
0.560081     1
0.576757     1
            ..
11.387378    1
12.080839    1
14.770174    1
15.190269    1
inf          1
Name: count, Length: 426, dtype: int64

## 결측치 확인

In [10]:
df['접근성지수'].isna().sum()

0

In [11]:
df['접근성지수'].unique()

array([ 2.67927229,  5.33177986,  7.69842394,  3.58512111,  3.51777325,
        0.64901569,  1.91738047,  6.03756493, 14.77017403,  8.45628143,
        3.95753142,  2.26585054,  3.25755379,  1.31297765,  2.26002028,
        2.45210656,  1.41820432, 11.32202284, 11.38737787, 15.19026923,
        6.43150487,  4.69121441,  5.92849933, 12.08083879,  4.16644777,
        1.81107681,  1.78850329,  0.41961566,  0.97367244,  0.75258102,
        0.81027334,  1.20033159,  1.53349192,  3.16222096,  6.38177525,
        1.69546979,  1.41777521,  1.88053709,  2.84448386,  1.42853412,
        4.76771198,  0.7453461 ,  0.87634405,  3.52400101,  2.10088389,
        3.38977981,  2.4896236 ,  1.01978914,  0.87901832,  1.04111981,
        1.89396643,  3.12563467,  3.52708285,  1.01541032,  1.94598278,
        1.56641125,  1.73583543,  2.00912098,  1.63830253,  2.83674354,
        2.01316132,  1.91046402,  3.7005502 ,  2.08986273,  3.03615302,
        1.93126551,  2.09598665,  3.32971889,  1.69019275,  2.71

In [12]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '행정동별유통업체수', '공공자전거대여소수', '인구minmax', '지하철minmax',
       '주차장minmax', '버스정류장minmax', '자전거대여소minmax', '접근성지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [13]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','계_명','지하철역_수','면수','행정동별버스정류장수', '공공자전거대여소수', '인구minmax','지하철minmax','주차장면수minmax','버스정류장minmax', '자전거대여소minmax', '접근성지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,계_명,지하철역_수,개소,행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,668,32,3,0.212126,0.000000,0.209007,0.294118,0.065217,2.679272
1,11110,종로구,1111053000,사직동,9362,2,362,27,12,0.169482,0.285714,0.111957,0.245098,0.260870,5.331780
2,11110,종로구,1111054000,삼청동,2576,0,343,16,5,0.045708,0.000000,0.105931,0.137255,0.108696,7.698424
3,11110,종로구,1111055000,부암동,9377,0,596,34,5,0.169755,0.000000,0.186172,0.313725,0.108696,3.585121
4,11110,종로구,1111056000,평창동,17658,0,1617,54,5,0.320797,0.000000,0.509990,0.509804,0.108696,3.517773


## 지수 파일 생성

In [14]:
df1.to_excel('접근성지수1.xlsx', index=False)

In [15]:
df2=read_excel('접근성지수1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,계_명,지하철역_수,개소,행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,668,32,3,0.212126,0.000000,0.209007,0.294118,0.065217,2.679272
1,11110,종로구,1111053000,사직동,9362,2,362,27,12,0.169482,0.285714,0.111957,0.245098,0.260870,5.331780
2,11110,종로구,1111054000,삼청동,2576,0,343,16,5,0.045708,0.000000,0.105931,0.137255,0.108696,7.698424
3,11110,종로구,1111055000,부암동,9377,0,596,34,5,0.169755,0.000000,0.186172,0.313725,0.108696,3.585121
4,11110,종로구,1111056000,평창동,17658,0,1617,54,5,0.320797,0.000000,0.509990,0.509804,0.108696,3.517773


## 이상치 `inf`, `-inf` 처리

In [16]:
# DataFrame(df2)에서 '접근성지수' 열을 선택
accessibility_series = df2['접근성지수']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '접근성지수' 열을 DataFrame에 할당
df2['접근성지수'] = accessibility_series

In [17]:
df2.to_excel("접근성지수1.xlsx", index=False)

In [18]:
df2 = read_excel("접근성지수1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,계_명,지하철역_수,개소,행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,668,32,3,0.212126,0.000000,0.209007,0.294118,0.065217,2.679272
1,11110,종로구,1111053000,사직동,9362,2,362,27,12,0.169482,0.285714,0.111957,0.245098,0.260870,5.331780
2,11110,종로구,1111054000,삼청동,2576,0,343,16,5,0.045708,0.000000,0.105931,0.137255,0.108696,7.698424
3,11110,종로구,1111055000,부암동,9377,0,596,34,5,0.169755,0.000000,0.186172,0.313725,0.108696,3.585121
4,11110,종로구,1111056000,평창동,17658,0,1617,54,5,0.320797,0.000000,0.509990,0.509804,0.108696,3.517773


## 지수 정규화

In [19]:
df2['접근성지수minmax'] = scaler.fit_transform(df2[['접근성지수']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,계_명,지하철역_수,개소,행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장minmax,버스정류장minmax,자전거대여소minmax,접근성지수,접근성지수minmax
0,11110,종로구,1111051500,청운효자동,11700,0,668,32,3,0.212126,0.000000,0.209007,0.294118,0.065217,2.679272,0.176381
1,11110,종로구,1111053000,사직동,9362,2,362,27,12,0.169482,0.285714,0.111957,0.245098,0.260870,5.331780,0.351000
2,11110,종로구,1111054000,삼청동,2576,0,343,16,5,0.045708,0.000000,0.105931,0.137255,0.108696,7.698424,0.506800
3,11110,종로구,1111055000,부암동,9377,0,596,34,5,0.169755,0.000000,0.186172,0.313725,0.108696,3.585121,0.236014
4,11110,종로구,1111056000,평창동,17658,0,1617,54,5,0.320797,0.000000,0.509990,0.509804,0.108696,3.517773,0.231581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,23303,3,1278,18,5,0.423759,0.428571,0.402474,0.156863,0.108696,2.587802,0.170359
422,11740,강동구,1174066000,성내제3동,22882,1,1241,15,6,0.416080,0.142857,0.390739,0.127451,0.130435,1.902235,0.125227
423,11740,강동구,1174068500,길동,45142,1,1734,36,7,0.822092,0.142857,0.547098,0.333333,0.152174,1.429843,0.094129
424,11740,강동구,1174069000,둔촌제1동,70,1,16,9,4,0.000000,0.142857,0.002220,0.068627,0.086957,0.000000,0.000000


## 최종 데이터 저장

In [21]:
df2.to_excel('접근성지수1.xlsx',index=False)